In [0]:
from pyspark.sql.types import *

json_schema = StructType([
    StructField("id", StringType()),
    StructField("name", StringType()),
    StructField("brewery_type", StringType()),
    StructField("city", StringType()),
    StructField("state", StringType()),
    StructField("country", StringType()),
    StructField("latitude", StringType()),
    StructField("longitude", StringType()),
    StructField("phone", StringType()),
    StructField("website_url", StringType())
])


In [0]:
from pyspark.sql.functions import from_json, col

# Ativa catálogo/schema
spark.sql("USE CATALOG projto_ambev")
spark.sql("USE SCHEMA silver")

# Lê os dados da bronze
df_bronze = spark.read.table("projto_ambev.bronze.breweries_json")

# Converte o campo JSON bruto (string) para struct
df_with_struct = df_bronze.withColumn("parsed", from_json(col("raw_data"), json_schema))

# Seleciona os campos da struct
df_silver = df_with_struct.select(
    "parsed.id", "parsed.name", "parsed.brewery_type", "parsed.city",
    "parsed.state", "parsed.country", "parsed.latitude", "parsed.longitude",
    "parsed.phone", "parsed.website_url"
)

# Salva na camada silver
df_silver.write.format("delta") \
    .mode("overwrite") \
    .saveAsTable("silver.breweries")

print("✅ Camada Silver criada com sucesso!")
